In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
from amftrack.pipeline.functions.post_processing.extract_study_zone import *
import pandas as pd
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.paths.directory import (
#     run_parallel,
#     find_state,
#     directory_scratch,
#     directory_project,
# )
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
plt.rcParams.update(
    {"font.family": "verdana", "font.weight": "normal", "font.size": 20}
)
from amftrack.plotutil import plot_node_skel
from amftrack.notebooks.validation.util import *
from amftrack.util.sys import *
from amftrack.notebooks.post_processing.util import *
import pickle
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
)

directory = directory_project
# directory = '/projects/0/einf914/agg/'
# directory = r"C:\Users\coren\Documents\PhD\Code\old_prince_data/"
update_analysis_info(directory)
analysis_info = get_analysis_info(directory)
# analysis_info['Plate']=analysis_info['Plate'].fillna(758)
# select = analysis_info.loc[analysis_info["Temp"] == "30to25"]

In [ ]:
select = analysis_info

In [ ]:
%load_ext memory_profiler

In [ ]:


from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)



In [ ]:
%%file mprun_demo.py
def sum_of_lists(N):
    %matplotlib widget
    import sys  
    sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
    from amftrack.pipeline.functions.post_processing.extract_study_zone import *
    import pandas as pd
    import ast
    from amftrack.plotutil import plot_t_tp1
    from scipy import sparse
    from datetime import datetime
    import pickle
    import scipy.io as sio
    from pymatreader import read_mat
    from matplotlib import colors
    import cv2
    import imageio
    import matplotlib.pyplot as plt
    import numpy as np
    from skimage.filters import frangi
    from skimage import filters
    from random import choice
    import scipy.sparse
    import os
    from skimage.feature import hessian_matrix_det
    from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
    from amftrack.notebooks.analysis.util import * 
    from scipy import stats
    from scipy.ndimage.filters import uniform_filter1d
    from collections import Counter
    from IPython.display import clear_output
    from amftrack.notebooks.analysis.data_info import *
    from IPython.core.interactiveshell import InteractiveShell
    InteractiveShell.ast_node_interactivity = "all"
    plt.rcParams.update({
        "font.family": "verdana",
    'font.weight' : 'normal',
    'font.size': 20})
    from amftrack.plotutil import plot_node_skel
    from amftrack.notebooks.validation.util import *
    from amftrack.pipeline.paths.directory import *
    from amftrack.util.sys import *
    from amftrack.notebooks.post_processing.util import *
    import pickle

    directory = directory_project
    update_analysis_info(directory)
    analysis_info = get_analysis_info(directory)
    select = analysis_info
    num = 0
    rows = [row for (index, row) in select.iterrows()]
    for index,row in enumerate(rows):
        path = f'{directory}{row["folder_analysis"]}'
        print(index,row["Plate"])
        try:
            a = np.load(f'{path}/center.npy')
        except:
            print(index,row["Plate"])
        if index == num:
            path_exp = f'{directory}{row["path_exp"]}'
            exp = pickle.load(open(path_exp, "rb"))
            exp.dates.sort()
            break

In [ ]:
plates = [
"3_20220426",
"12_20220502",
"13_20220422",
"16_20220419",
"21_20220502",
"480_20221205",
"28_20230227",
"206_20230303",
"202_20230314",
"218_20230227",
"219_20230307",
"229_20230330",
# "240_20230328"
]

In [ ]:
plate = "3_20220426"
rows = [row for (index, row) in select.iterrows()]
for index, row in enumerate(rows):
    path = f'{directory}{row["folder_analysis"]}'
    try:
        a = np.load(f"{path}/num_trunk.npy")
    except:
        print(index, row["Plate"])
    if row["unique_id"] == plate:
        path_exp = f'{directory}{row["path_exp"]}'
        exp = pickle.load(open(path_exp, "rb"))
        exp.dates.sort()
        break

In [ ]:
exp

In [ ]:
load_graphs(exp, directory, indexes=range(0, 1))
# exp.nx_graph

In [ ]:
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)

load_study_zone(exp)

In [ ]:
plt.close("all")
i = 0
dist = 150
radius = 1000
compress = 5
dr_orth, dr_center = get_study_zone(exp, dist, radius, i)

In [ ]:
extract_study_zone(dr_orth, dr_center, exp)

In [ ]:
def save_study_zone(exp):
    exp.center = np.save(f"{exp.save_location}/center.npy", exp.center)
    exp.orthog = np.save(f"{exp.save_location}/orthog.npy", exp.orthog)

In [ ]:
save_study_zone(exp)

In [ ]:
save_study_zone(dr_orth, dr_center, exp, i)

In [ ]:
exp.ts

In [ ]:
dist = 150
radius = 1000

In [ ]:
exp

In [ ]:
end = 27
plt.close("all")
compress = 25
date = exp.dates[end]
directory_name = get_dirname(date, exp.folders)
path_snap = exp.directory + directory_name
skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
Rot = skel["R"]
trans = skel["t"]
skelet = skel["skeleton"]
output = skelet.todense()
kernel = np.ones((5, 5), np.uint8)
output = cv2.dilate(output.astype(np.uint8), kernel, iterations=3)
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111)
x, y = np.array(exp.center)[0] // compress, np.array(exp.center)[1] // compress
circle = plt.Circle((x, y), radius, alpha=0.3, color="red")
pos_line = np.array((x, y)) + dist * exp.orthog
orth_direct = np.array([exp.orthog[1], -exp.orthog[0]])
extension = 1000
deb_line = pos_line + extension * orth_direct
end_line = pos_line - extension * orth_direct
line = pltlines.Line2D(
    (deb_line[0], end_line[0]), (deb_line[1], end_line[1]), color="red"
)
ax.add_patch(circle)
ax.add_line(line)
ax.imshow(
    cv2.resize(output, (output.shape[1] // compress, output.shape[0] // compress)),
    alpha=1,
)

In [ ]:
exp.reach_out = 100

In [ ]:
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, path_code

path_code

In [ ]:
# path = f'{directory}{row["folder_analysis"]}'
# np.save(f"{path}/center", exp.center)
# np.save(f"{path}/orthog", exp.orthog)
np.save(f"{path}/reach_out", exp.reach_out)

In [ ]:
plt.close("all")
frame = exp.reach_out
# frame = 35
# plot_raw2(exp, frame)

In [ ]:
exp.num_trunk = 3

In [ ]:
path = f'{directory}{row["folder_analysis"]}'
np.save(f"{path}/num_trunk", exp.num_trunk)

In [ ]:
exp.labeled = True
load_graphs(exp, indexes=[0])

In [ ]:
exp.folders

In [ ]:
nx_graph = exp.nx_graph[60]

In [ ]:
t = 0
nx_graph = exp.nx_graph[t]
threshold = 0.1
S = [nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph)]
selected = [
    g for g in S if g.size(weight="weight") * len(g.nodes) / 10**6 >= threshold
]
selected
polys = Polygon()
if len(selected) >= 0:
    area_max = 0
    for g in selected:
        nodes = np.array(
            [
                node.pos(t)
                for node in exp.nodes
                if node.is_in(t)
                and np.all(is_in_study_zone(node, t, 1000, 150))
                and (node.label in g.nodes)
            ]
        )
        if len(nodes) > 3:
            hull = spatial.ConvexHull(nodes)
            poly = Polygon([nodes[vertice] for vertice in hull.vertices])
            area_hull = poly.area * 1.725**2 / (1000**2)
            polys = polys.union(poly)

In [ ]:
date = exp.dates[70]
directory_name = get_dirname(date, exp.plate)
path_snap = exp.directory + directory_name
suffix = "/Analysis/nx_graph_pruned_labeled.p"
path_save = path_snap + suffix
(g, pos) = pickle.load(open(path_save, "rb"))

In [ ]:
g.nodes

In [ ]:
path

In [ ]:
def plot_raw2(exp, t, figsize=(10, 9)):
    date = exp.dates[t]
    directory_name = get_dirname(date, exp.plate)
    path_snap = exp.directory + directory_name
    im = read_mat(path_snap + "/Analysis/raw_image.mat")["raw"]
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    ax.imshow(im, cmap="gray")

In [ ]:
Node(126416, exp).show_source_image(90, 90)

In [ ]:
Edge(Node(126416, exp), Node(141897, exp)).show_source_image(90, 90)

In [ ]:
len(Edge(Node(126416, exp), Node(141897, exp), exp).pixel_list(90))

In [ ]:
def get_width_info(experiment, t, resolution=50, skip=False):
    print(not skip)
    edge_width = {}
    graph = experiment.nx_graph[t]
    #     print(len(list(graph.edges)))
    # print(len(graph.edges))
    print(len(graph.edges))
    # list_edges = [choice(list(graph.edges)) for i in range(100)]
    for edge in graph.edges:
        if not skip:
            # print(edge)
            edge_exp = Edge(
                Node(edge[0], experiment), Node(edge[1], experiment), experiment
            )
            mean = np.mean(list(get_width_edge(edge_exp, resolution, t).values()))
            #         print(np.mean(list(get_width_edge(edge_exp,resolution,t).values())))
            edge_width[edge] = mean
            # print(mean)
        else:
            # Maybe change to Nan if it doesnt break the rest
            edge_width[edge] = 40
    return edge_width

In [ ]:
%%time
get_width_info(exp, 90, resolution=50)

In [ ]:
from skimage.measure import profile_line
from amftrack.notebooks.analysis.util import *
from scipy.optimize import curve_fit
import numpy as np

a = 2.3196552
from scipy import special


def func2(x, lapse, lapse2, c, d, e):
    return (
        -c * (special.erf(e * (x - lapse)) - special.erf(e * (x - lapse - lapse2))) + d
    )


def func3(x, lapse, lapse2, c, d, e, lapse4):
    return (
        -c * (special.erf(e * (x - lapse)) - special.erf(e * (x - (lapse + lapse2))))
        + d
        + c
        * (
            special.erf(e * (x - (lapse + lapse2)))
            - special.erf(e * (x - (lapse + lapse2 + lapse4)))
        )
    )


def func4(x, lapse, lapse2, c, d, e, lapse4):
    return (
        -c * (special.erf(e * (x - lapse)) - special.erf(e * (x - (lapse + lapse2))))
        + d
        + c * (special.erf(e * (x - (lapse - lapse4))) - special.erf(e * (x - (lapse))))
    )


def func5(x, sigma, mean, fact, offset):
    return -fact * np.exp(-((x - mean) ** 2) / sigma**2) + offset


def func5(x, sigma, mean, fact, offset):
    return -fact * np.exp(-((x - mean) ** 2) / sigma**2) + offset


def get_source_image(experiment, pos, t, local, force_selection=None):
    x, y = pos[0], pos[1]
    ims, posimg = experiment.find_image_pos(x, y, t, local)
    if force_selection is None:
        dist_border = [
            min([posimg[1][i], 3000 - posimg[1][i], posimg[0][i], 4096 - posimg[0][i]])
            for i in range(posimg[0].shape[0])
        ]
        j = np.argmax(dist_border)
    else:
        dist_last = [
            np.linalg.norm(
                np.array((posimg[1][i], posimg[0][i])) - np.array(force_selection)
            )
            for i in range(posimg[0].shape[0])
        ]
        j = np.argmin(dist_last)
    return (ims[j], (posimg[1][j], posimg[0][j]))


def get_width_pixel(
    edge,
    index,
    im,
    pivot,
    before,
    after,
    t,
    size=20,
    width_factor=60,
    averaging_size=100,
    threshold_averaging=10,
):
    imtab = im
    #     print(imtab.shape)
    #     print(int(max(0,pivot[0]-averaging_size)),int(pivot[0]+averaging_size))
    threshold = np.mean(
        imtab[
            int(max(0, pivot[0] - averaging_size)) : int(pivot[0] + averaging_size),
            int(max(0, pivot[1] - averaging_size)) : int(pivot[1] + averaging_size),
        ]
        - threshold_averaging
    )
    orientation = np.array(before) - np.array(after)
    perpendicular = (
        [1, -orientation[0] / orientation[1]] if orientation[1] != 0 else [0, 1]
    )
    perpendicular_norm = np.array(perpendicular) / np.sqrt(
        perpendicular[0] ** 2 + perpendicular[1] ** 2
    )
    point1 = np.around(np.array(pivot) + width_factor * perpendicular_norm)
    point2 = np.around(np.array(pivot) - width_factor * perpendicular_norm)
    point1 = point1.astype(int)
    point2 = point2.astype(int)
    p = profile_line(imtab, point1, point2, mode="constant")
    xdata = np.array(range(len(p)))
    ydata = np.array(p)
    #     fig = plt.figure()
    #     ax = fig.add_subplot(111)
    #     ax.plot(xdata,ydata)
    #     ax.plot(xdata, func5(xdata, *popt0), 'g-')
    # try:
    #     raise RuntimeError
    # #     p00=[10,60,60,160]
    # #     popt0, pcov = curve_fit(func5, xdata, ydata,bounds = ([0,0,0,0],4*[np.inf]),p0=p00)
    # #     p0a=[60,10,100,180,0.1]
    # #     popt1, pcov = curve_fit(func2, xdata, ydata,bounds = ([0,0,0,0,0],[120,120,200]+2*[np.inf]),p0=p0a)
    # #     p0b=list(popt1)+[10]
    # #     popt2, pcov = curve_fit(func3, xdata, ydata,bounds = ([0,0,0,0,0,0],[120,120,200]+2*[np.inf]+[120]),p0=p0b)
    # #     residuals = ydata- func3(xdata, *popt2)
    # #     ss_res = np.sum(residuals**2)
    # #     ss_tot = np.sum((ydata-np.mean(ydata))**2)
    # #     r_squared1 = 1 - (ss_res / ss_tot)
    # #     popt3, pcov = curve_fit(func4, xdata, ydata,bounds = ([0,0,0,0,0,0],[120,120,200]+2*[np.inf]+[120]),p0=p0b)
    # #     residuals = ydata- func4(xdata, *popt3)
    # #     ss_res = np.sum(residuals**2)
    # #     ss_tot = np.sum((ydata-np.mean(ydata))**2)
    # #     r_squared2 = 1 - (ss_res / ss_tot)
    # # #     ax.plot(xdata, func2(xdata, *popt1), 'r-')
    # #     if r_squared1>r_squared2:
    # # #         ax.plot(xdata, func3(xdata, *popt2), 'b-')
    # #         popt=popt2
    # #     else:
    # # #         ax.plot(xdata, func4(xdata, *popt3), 'b-')
    # #         popt=popt3
    # #     background = popt[3]
    # except RuntimeError:
    #     print('failed')
    background = np.mean(
        (np.mean(p[: width_factor // 6]), np.mean(p[-width_factor // 6 :]))
    )
    #     print(popt[3],popt0[3])
    #     width_pix = popt0[0]*popt0[2]
    width_pix = -np.sum(
        (np.log10(np.array(p) / background) <= 0) * np.log10(np.array(p) / background)
    )
    #     print(width_pix)
    #     p0=[165,100,165,45,10,10,10]
    #     popt, pcov = curve_fit(func, xdata, ydata,bounds = ([-np.inf,-np.inf,-np.inf,-np.inf,0,0,0],np.inf),p0=p0)
    #     width_pix = popt[-2]
    #     ax.plot(xdata, func(xdata, *popt), 'r-')
    #     derivative = [p[i+1]-p[i] for i in range(len(p)-1)]
    #     fig = plt.figure()
    #     ax = fig.add_subplot(111)
    #     ax.plot([np.mean(derivative[5*i:5*i+5]) for i in range(len(derivative)//5)])
    #     problem=False
    #     arg = len(p)//2
    #     if p[arg]>threshold:
    #         arg = np.argmin(p)
    # #     we_plot=randrange(1000)
    #     while  p[arg]<=threshold:
    #         if arg<=0:
    # #             we_plot=50
    #             problem=True
    #             break
    #         arg-=1
    #     begin = arg
    #     arg = len(p)//2
    #     if p[arg]>threshold:
    #         arg = np.argmin(p)
    #     while  p[arg]<=threshold:
    #         if arg>=len(p)-1:
    # #             we_plot=50
    #             problem=True
    #             break
    #         arg+=1
    #     end = arg
    # #     print(end-begin,threshold)
    #     print(np.linalg.norm(point1-point2),len(p),width_pix)
    return a * np.sqrt(max(0, np.linalg.norm(point1 - point2) * (width_pix) / len(p)))


def get_width_edge(edge, resolution, t, local=False, threshold_averaging=10):
    pixel_conversion_factor = 1.725
    pixel_list = edge.pixel_list(t)
    pixels = []
    indexes = []
    source_images = []
    poss = []
    widths = {}
    if len(pixel_list) > 3 * resolution:
        for i in range(0, len(pixel_list) // resolution):
            index = i * resolution
            indexes.append(index)
            pixel = pixel_list[index]
            pixels.append(pixel)
            source_img, pos = get_source_image(edge.experiment, pixel, t, local)
            source_images.append(source_img)
            poss.append(pos)
    else:
        indexes = [0, len(pixel_list) // 2, len(pixel_list) - 1]
        for index in indexes:
            pixel = pixel_list[index]
            pixels.append(pixel)
            source_img, pos = get_source_image(edge.experiment, pixel, t, local)
            source_images.append(source_img)
            poss.append(pos)
    #     print(indexes)
    for i, index in enumerate(indexes[1:-1]):
        source_img = source_images[i + 1]
        pivot = poss[i + 1]
        _, before = get_source_image(edge.experiment, pixels[i], t, local, pivot)
        _, after = get_source_image(edge.experiment, pixels[i + 2], t, local, pivot)
        #         plot_t_tp1([0,1,2],[],{0 : pivot,1 : before, 2 : after},None,source_img,source_img)
        width = get_width_pixel(
            edge,
            index,
            source_img,
            pivot,
            before,
            after,
            t,
            threshold_averaging=threshold_averaging,
        )
        #         print(width*pixel_conversion_factor)
        widths[pixel_list[index]] = width * pixel_conversion_factor
    #         if i>=1:
    #             break
    edge.experiment.nx_graph[t].get_edge_data(edge.begin.label, edge.end.label)[
        "width"
    ] = widths
    return widths

In [ ]:
path = f'{directory}{row["folder_analysis"]}'
exp.pickle_save(path)

In [ ]:
select

In [ ]:
plt.close("all")
compress = 25
date = exp.dates[i]
directory_name = get_dirname(date, exp.plate)
path_snap = exp.directory + directory_name
skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
Rot = skel["R"]
trans = skel["t"]
skelet = skel["skeleton"]
output = skelet.todense()
kernel = np.ones((5, 5), np.uint8)
output = cv2.dilate(output.astype(np.uint8), kernel, iterations=3)
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111)
ax.imshow(
    cv2.resize(output, (output.shape[1] // compress, output.shape[0] // compress)),
    alpha=1,
)
plt.scatter(exp.center[0] // compress, exp.center[1] // compress)

In [ ]:
Node(77, exp)

In [ ]:
is_in_study_zone(Node(223, exp), t)

In [ ]:
np.all((True, False))

In [ ]:
t = 0
compress = 25
center = np.array(exp.center)
poss = exp.positions[t]
x0, y0 = exp.center
direction = exp.orthog
pos_line = np.array((x0, y0)) + dist * compress * direction
x_line, y_line = pos_line[0], pos_line[1]
orth_direct = np.array([direction[1], -direction[0]])
x_orth, y_orth = orth_direct = orth_direct[0], orth_direct[1]
a = y_orth / x_orth
b = y_line - a * x_line
nodes_exclude = []
for node in exp.nx_graph[t].nodes:
    dist_center = np.linalg.norm(poss[node] - center)
    y, x = poss[node]
    if dist_center > radius * compress or a * x + b < y:
        nodes_exclude.append(node)

In [ ]:
plt.close("all")
compress = 25
date = exp.dates[i]
directory_name = get_dirname(date, exp.plate)
path_snap = exp.directory + directory_name
skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
Rot = skel["R"]
trans = skel["t"]
skelet = skel["skeleton"]
output = skelet.todense()
kernel = np.ones((5, 5), np.uint8)
output = cv2.dilate(output.astype(np.uint8), kernel, iterations=3)
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111)
ax.imshow(
    cv2.resize(output, (output.shape[1] // compress, output.shape[0] // compress)),
    alpha=1,
)
plt.scatter(exp.center[0] // compress, exp.center[1] // compress)
plt.scatter(x_line // compress, y_line // compress)
for node in exp.nx_graph[t].nodes:
    if node not in nodes_exclude:
        s = plt.scatter(*np.flip(poss[node]) // compress)